## **Load libraries and data**

In [1]:
!pip install --upgrade pythainlp
!pip install pyLDAvis

     |████████████████████████████████| 11.0 MB 7.1 MB/s eta 0:00:01    |██████████████████▏             | 6.2 MB 1.8 MB/s eta 0:00:03
     |████████████████████████████████| 186 kB 6.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 1.8 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 23.9 MB 7.7 MB/s eta 0:00:01    |███████████▍                    | 8.5 MB 1.3 MB/s eta 0:00:12
     |████████████████████████████████| 16.0 MB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 10.5 MB 6.3 MB/s eta 0:00:01    |█████▎                          | 1.7 MB 8.6 MB/s eta 0:00:02
     |████████████████████████████████| 57 kB 3.1 MB/s eta 0:00:01
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=224a7b05745b73b8f38151a19700bd976895b922683ff154e5ec36254e3f

In [3]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [5]:
df = pd.read_csv('CustomerReviews.csv')

In [8]:
df.head()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating
0,1,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,7b16469831074f7abc7824745ee75212,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,5.0
1,2,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,pakkaramonpondej,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่งดี,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,5.0
2,3,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,saanowy,ชาบูพรีเมี่ยมสไตล์ญี่ปุ่น เนื้อดีมากกก,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,4.0
3,4,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ployynp,เนื้อดี ไอติมดี คุ้มค่าดี,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4.0
4,5,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,665a902b335b434ab489f00f2fbb477e,อาหารมีคุณภาพ บริการดีค่ะ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,5.0


Tokenize Words

In [30]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '\n', 'ร้าน', '(', ')' , '           ','ที่', 'ค่ะ', 'ครับ', 'ๆ', '-', '""']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [31]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [32]:
df.tail()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,Review_tokenized
16,17,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,ployynp,บุฟเฟ่ต์ชาบูและพิซซ่าไม่อั้นในราคา 199 บาท เน้...,หลังจากที่เคยลองสาขายูเนี่ยนมอลล์ไป รอบนี้มาที...,4.0,"หลังจากที่,ลอง,สาขา,ยู,นม,อลล์,รอบ,สาขา,เดอะ,ม..."
17,18,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,27a91236fe5e4559a4f097c97a480781,ร้านบุฟเฟ่ต์ ราคามิตรภาพ อยู่ชั้น4 ติดโรงหนัง ...,ร้านบุฟเฟ่ต์ที่มีโปรโมชั่นหัวละ199บาท ไม่รวมน้...,4.0,"บุฟเฟ่ต์,โปรโมชั่น,หัว,199,บาท,น้ำ,VAT,ทาน,ธรร..."
18,19,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,0b81d251e4db486f9bcdba73b374ed99,ของหลากหลาย ปนๆ งงๆ นิดหน่อย,เคยรู้จักร้านนี้จากที่ union mall ไม่เคยได้ลอง...,3.0,"รู้จัก,union,mall,ลอง,กิน,จำได้,ขึ้นใจ,ชื่อ,😆,..."
19,20,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,40e0e087f3914fd49a8933b5a29936ca,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,5.0,"อร่อย,คุ้มค่า,สมราคา,บุ,ฟเฟ่,หมู,ผัก,น้ำ,จบ,25..."
20,21,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,41841cb99ea243a3a8d4b006e946c586,แม้จะแปลกบ้าง แต่ก็ถือว่าอยู่ในเกณฑ์ที่ดี มีอา...,ก็ตามที่เขียนเลยครับ ว่า ถ้าจะจ่ายในราคา 199 บ...,NaN,"จ่าย,ราคา,199,บาท,จ่าย,เงินสด,ราคา,น้ำ,VAT,7,%..."


## **Create Dictionary**

In [33]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [34]:
print(dictionary.token2id.keys())

dict_keys(['1', '4', '560', 'กิน', 'ข้างนอก', 'ความต้องการ', 'คอ', 'คุณภาพ', 'คุ้ม', 'จำกัด', 'ซุป', 'ดำ', 'ดี', 'ดีมาก', 'ถาด', 'ถาม', 'น่ารัก', 'น้ำ', 'น้ำจิ้ม', 'บริการ', 'บาท', 'ประทับใจ', 'พนักงาน', 'พอกับ', 'ภาพ', 'รสชาติ', 'ราคา', 'สรุป', 'สะอาดสะอ้าน', 'สัน', 'หลากหลาย', 'อร่อย', 'อันนี้', 'อาหาร', 'เคลม', 'เซ็ท', 'เต้าเจี้ยว', 'เนื้อ', 'เป็ด', 'เผ็ด', 'แบบนี้', 'แปลก', 'แฟน', 'โดยรวม', 'โมจิ', 'ใส่ใจ', 'ไม่ต้อง', 'ไอติม', '\u200b', '21', '30', '9', 'CTW', 'Terminal', 'กา', 'การงด', 'คิดถึง', 'ชาบู', 'ญี่ปุ่น', 'ฐาน', 'ทะยอย', 'ทำงาน', 'ที่ทาง', 'นาที', 'นึกถึง', 'พนักงานบริการ', 'พระราม', 'พรีเมียม', 'พาราไดซ์', 'มาตรา', 'มีโอ', 'รายละเอียด', 'ลูกค้า', 'สมาทาน', 'สาขา', 'สู้', 'ออร์เดอร์', 'เครื่องดื่ม', 'เค้า', 'เจอ', 'เป็นกันเอง', 'เหวี่ยง', 'แจ้ง', 'โม', 'โม่', 'โอเค', '', '++', '2', '555', '600', ':', '::', 'กก', 'กรุบ', 'กั้น', 'ของหวาน', 'คนละ', 'คาว', 'คุ', 'คุ้มค่า', 'จาน', 'จิ้ม', 'จืด', 'ฉาก', 'ชอบ', 'ชาเขียว', 'ชิ้น', 'ซอย', 'ซอส', 'ตก', 'ตะ', 'ตัก', 'ตามน้ำ', 'ต่าง

In [35]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]

In [36]:
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [37]:
num_topics = 20
chunksize = 1000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 370 ms, sys: 8.33 ms, total: 378 ms
Wall time: 390 ms


In [38]:
gensimvis.prepare(model, gensim_corpus, dictionary, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
8      -10.878068   10.538728       1        1  20.647670
11     -60.884026  -75.591820       2        1  14.125609
13      -2.336119  -53.289074       3        1  11.769649
19     -61.385319  -12.914634       4        1   9.718373
17     -28.737743  126.296997       5        1   7.972053
9        1.066606 -123.552811       6        1   6.100319
4       45.140720   -7.947869       7        1   5.339940
1     -140.795975  -98.530548       8        1   4.295580
6       61.531681  -83.240562       9        1   3.812010
0      -72.563057 -140.396881      10        1   3.345648
2       15.984491   63.578186      11        1   2.973683
15    -160.851624   68.945145      12        1   2.848453
3       85.961830   51.922935      13        1   2.633494
18     -99.020836  109.936348      14        1   2.206676
5     -176.764130   -8.681684      15        1   2.089981
14    -117.189880  -37.420200      16        1   0.025367
7      108.829918  -22.946203      17        1   0.024302
16      47.588097  121.934814      18        1   0.023731
12    -106.467834   30.176889      19        1   0.023731
10     -50.613804   59.328842      20        1   0.023731, topic_info=           Term       Freq      Total Category  logprob  loglift
48            ​   9.000000   9.000000  Default    30.00  30.0000
91            :  12.000000  12.000000  Default    29.00  29.0000
86                7.000000   7.000000  Default    28.00  28.0000
3           กิน  18.000000  18.000000  Default    27.00  27.0000
17          น้ำ  21.000000  21.000000  Default    26.00  26.0000
..          ...        ...        ...      ...      ...      ...
25       รสชาติ   0.000543  12.237626  Topic20    -6.42  -1.6774
26         ราคา   0.000543  14.388841  Topic20    -6.42  -1.8394
27         สรุป   0.000543   2.066330  Topic20    -6.42   0.1013
28  สะอาดสะอ้าน   0.000543   1.223389  Topic20    -6.42   0.6255
29          สัน   0.000543   1.859032  Topic20    -6.42   0.2070

[1000 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
86        3  0.644668         
86       13  0.257867         
247       9  0.869580     !!!!
336       1  0.307367        "
336       6  0.614733        "
...     ...       ...      ...
270       9  0.869580  ไอศครีม
48        7  0.997385        ​
246       5  0.771642        –
582       6  0.801284        😆
583       6  0.801282        🤣

[789 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 12, 14, 20, 18, 10, 5, 2, 7, 1, 3, 16, 4, 19, 6, 15, 8, 17, 13, 11])